In [13]:
import torch
from torchvision import transforms
import torch.nn as nn

In [14]:
import torchvision.datasets as datasets
train_dataset = datasets.MNIST(root='../../data/',
                               train=True, 
                               transform=transforms.ToTensor(),
                               download=True)                                             
test_dataset = datasets.MNIST(root='../../data/',
                              train=False, 
                              transform=transforms.ToTensor())

In [15]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=32, 
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=32, 
                                          shuffle=False)

In [16]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, 
    num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                            batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        h = torch.zeros(self.num_layers, x.size(0), 
                        self.hidden_size).to(device) 
        c = torch.zeros(self.num_layers, x.size(0), 
                        self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h, c))  
        
        out = self.fc(out[:, -1, :])
        return out

In [17]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)
class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)
def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [18]:
device = get_default_device()

In [19]:
# Hyper parameters
learning_rate = 0.001
sequence_length = 28
hidden_size = 128
num_classes = 10
batch_size = 64
input_size = 28
num_layers = 2
num_epochs = 3
model = RNN(input_size, hidden_size, num_layers, num_classes)
to_device(model, device)

RNN(
  (lstm): LSTM(28, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=128, out_features=10, bias=True)
)

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [21]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/3], Step [100/1875], Loss: 1.2317
Epoch [1/3], Step [200/1875], Loss: 0.4318
Epoch [1/3], Step [300/1875], Loss: 0.4514
Epoch [1/3], Step [400/1875], Loss: 0.3029
Epoch [1/3], Step [500/1875], Loss: 0.3350
Epoch [1/3], Step [600/1875], Loss: 0.1149
Epoch [1/3], Step [700/1875], Loss: 0.1838
Epoch [1/3], Step [800/1875], Loss: 0.4131
Epoch [1/3], Step [900/1875], Loss: 0.0448
Epoch [1/3], Step [1000/1875], Loss: 0.2076
Epoch [1/3], Step [1100/1875], Loss: 0.1394
Epoch [1/3], Step [1200/1875], Loss: 0.1046
Epoch [1/3], Step [1300/1875], Loss: 0.0463
Epoch [1/3], Step [1400/1875], Loss: 0.0962
Epoch [1/3], Step [1500/1875], Loss: 0.0090
Epoch [1/3], Step [1600/1875], Loss: 0.1411
Epoch [1/3], Step [1700/1875], Loss: 0.0466
Epoch [1/3], Step [1800/1875], Loss: 0.3137
Epoch [2/3], Step [100/1875], Loss: 0.1684
Epoch [2/3], Step [200/1875], Loss: 0.0105
Epoch [2/3], Step [300/1875], Loss: 0.3014
Epoch [2/3], Step [400/1875], Loss: 0.1627
Epoch [2/3], Step [500/1875], Loss: 0.1188
Ep

In [22]:
# Evaluate the model
model.eval()
with torch.no_grad():
    right = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, 
                 input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        right += (predicted == labels).sum().item()
print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * right / total))

Test Accuracy of the model on the 10000 test images: 98.33 %
